In [357]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from XGBoost import XGBoost

In [358]:
df = pd.read_csv('hotel_bookings.csv')
pd.set_option('display.max_columns', None)

In [359]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [360]:
df.isnull().sum().sort_values(ascending=False).head()

company               112593
agent                  16340
country                  488
children                   4
reserved_room_type         0
dtype: int64

In [361]:
df.describe()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119386.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,103050.000000,6797.000000,119390.000000,119390.000000,119390.000000,119390.000000
mean,0.370416,104.011416,2016.156554,27.165173,15.798241,0.927599,2.500302,1.856403,0.103890,0.007949,0.031912,0.087118,0.137097,0.221124,86.693382,189.266735,2.321149,101.831122,0.062518,0.571363
std,0.482918,106.863097,0.707476,13.605138,8.780829,0.998613,1.908286,0.579261,0.398561,0.097436,0.175767,0.844336,1.497437,0.652306,110.774548,131.655015,17.594721,50.535790,0.245291,0.792798
min,0.000000,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000,0.000000,-6.380000,0.000000,0.000000
25%,0.000000,18.000000,2016.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,62.000000,0.000000,69.290000,0.000000,0.000000
50%,0.000000,69.000000,2016.000000,28.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,179.000000,0.000000,94.575000,0.000000,0.000000
75%,1.000000,160.000000,2017.000000,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,229.000000,270.000000,0.000000,126.000000,0.000000,1.000000
max,1.000000,737.000000,2017.000000,53.000000,31.000000,19.000000,50.000000,55.000000,10.000000,10.000000,1.000000,26.000000,72.000000,21.000000,535.000000,543.000000,391.000000,5400.000000,8.000000,5.000000


In [362]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [363]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [364]:
df.isnull().sum().sort_values(ascending=False).head()

company               112593
agent                  16340
country                  488
children                   4
reserved_room_type         0
dtype: int64

In [365]:
# Menghitung modus kolom 'agent'
mode_agent = df['agent'].mode()[0]  # Mengambil nilai modus pertama

# Mengisi nilai NaN dengan modus
df['agent'] = df['agent'].fillna(mode_agent)

In [366]:
df.isnull().sum().sort_values(ascending=False).head()

company               112593
country                  488
children                   4
agent                      0
reserved_room_type         0
dtype: int64

In [367]:
# Menghitung modus kolom 'country'
mode_country = df['country'].mode()[0]  # Mengambil nilai modus pertama

# Mengisi nilai NaN dengan modus
df['country'] = df['country'].fillna(mode_country)

In [368]:
df['children'] = df['children'].fillna(0)

In [369]:
df.isnull().sum().sort_values(ascending=False).head()

company                      112593
hotel                             0
is_canceled                       0
reservation_status                0
total_of_special_requests         0
dtype: int64

In [370]:
# Drop duplicate

print("Baris sebelum duplicate:", len(df))
df.drop_duplicates(inplace=True)
print("Rows setelah duplicate", len(df))
df.reset_index(drop=True, inplace=True)

Baris sebelum duplicate: 119390
Rows setelah duplicate 87376


In [371]:
# Terdapat 0 tamu

zero_guests_df = df.loc[df["adults"]
                   + df["children"]
                   + df["babies"] == 0].index

print(len(zero_guests_df))

166


In [372]:
# Drop 0 tamu

print("Baris sebelum drop 0 tamu:", len(df))
df.drop(df.index[zero_guests_df], axis=0, inplace=True)
print("Baris setelah drop 0 tamu:", len(df))
df.reset_index(drop=True, inplace=True)

Baris sebelum drop 0 tamu: 87376
Baris setelah drop 0 tamu: 87210


In [373]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87210 entries, 0 to 87209
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           87210 non-null  object 
 1   is_canceled                     87210 non-null  int64  
 2   lead_time                       87210 non-null  int64  
 3   arrival_date_year               87210 non-null  int64  
 4   arrival_date_month              87210 non-null  object 
 5   arrival_date_week_number        87210 non-null  int64  
 6   arrival_date_day_of_month       87210 non-null  int64  
 7   stays_in_weekend_nights         87210 non-null  int64  
 8   stays_in_week_nights            87210 non-null  int64  
 9   adults                          87210 non-null  int64  
 10  children                        87210 non-null  float64
 11  babies                          87210 non-null  int64  
 12  meal                            

In [374]:
# Membuat countries yang banyak menjadi 10 saja dan sisanya others

n_countries = 10
most_freq_countries = df['country'].value_counts()[:n_countries].index
df['country'].value_counts()[:n_countries]

df['country'] = df['country'].apply(lambda x: x if x in most_freq_countries else 'Others')

In [375]:
# Membuat company yang banyak menjadi 10 saja dan sisanya others

n_agents = 10
most_freq_agent = df['agent'].value_counts()[:n_agents].index
df['agent'].value_counts()[:n_agents]

df['agent'] = df['agent'].apply(lambda x: x if x in most_freq_agent else 'Others')

C:\Users\fadhl\AppData\Local\Temp\ipykernel_7248\2463387029.py:4: FutureWarning: The behavior of obj[i:j] with a float-dtype index is deprecated. In a future version, this will be treated as positional instead of label-based. For label-based slicing, use obj.loc[i:j] instead
  most_freq_agent = df['agent'].value_counts()[:n_agents].index
C:\Users\fadhl\AppData\Local\Temp\ipykernel_7248\2463387029.py:5: FutureWarning: The behavior of obj[i:j] with a float-dtype index is deprecated. In a future version, this will be treated as positional instead of label-based. For label-based slicing, use obj.loc[i:j] instead
  df['agent'].value_counts()[:n_agents]


In [376]:
df['agent'].head()

0       9.0
1       9.0
2       9.0
3    Others
4     240.0
Name: agent, dtype: object

In [377]:
# Definisikan mapping bulan ke kuartal
month_to_quarter = {
    'January': 'Quarter_1',
    'February': 'Quarter_1',
    'March': 'Quarter_1',
    'April': 'Quarter_2',
    'May': 'Quarter_2',
    'June': 'Quarter_2',
    'July': 'Quarter_3',
    'August': 'Quarter_3',
    'September': 'Quarter_3',
    'October': 'Quarter_4',
    'November': 'Quarter_4',
    'December': 'Quarter_4'
}

# Buat kolom baru dengan mapping
df['arrival_date_quarter'] = df['arrival_date_month'].map(month_to_quarter)

In [378]:
# Buat kolom baru 'total_orang' dengan menjumlahkan 'adults', 'children', dan 'babies'
df['total_orang'] = df['adults'] + df['children'] + df['babies']

In [379]:
df = df.drop('company', axis=1)

In [380]:
# Fungsi untuk mengubah kategori menjadi numerik
def categorical_to_numeric(df):
    for column in df.select_dtypes(include=['object']).columns:
        # Mengambil kategori unik dari kolom
        unique_categories = df[column].unique()
        
        # Membuat dictionary mapping kategori ke angka
        category_map = {category: idx for idx, category in enumerate(unique_categories)}
        
        # Mengubah kolom berdasarkan mapping
        df[column] = df[column].map(category_map)
    
    return df

# Mengubah semua kolom kategorikal menjadi numerik
df = categorical_to_numeric(df)

# Menampilkan DataFrame setelah kategori diubah menjadi numerik
print("\nDataframe setelah kategorikal menjadi numerik:")


Dataframe setelah kategorikal menjadi numerik:


In [381]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_date_quarter,total_orang
0,0,0,342,2015,0,27,1,0,0,2,0.0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0.0,0,0,0,0,0,2.0
1,0,0,737,2015,0,27,1,0,0,2,0.0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0.0,0,0,0,0,0,2.0
2,0,0,7,2015,0,27,1,0,1,1,0.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,75.0,0,0,0,1,0,1.0
3,0,0,13,2015,0,27,1,0,1,1,0.0,0,0,1,1,1,0,0,0,1,1,0,0,1,0,0,75.0,0,0,0,1,0,1.0
4,0,0,14,2015,0,27,1,0,2,2,0.0,0,0,1,2,2,0,0,0,1,1,0,0,2,0,0,98.0,0,1,0,2,0,2.0


In [382]:
# Hitung jumlah setiap nilai unik di kolom 'agent'
value_counts = df['agent'].value_counts()

# Hitung persentase setiap nilai
percentage = (value_counts / len(df)) * 100

# Gabungkan nilai dan persentase untuk dilihat bersama
result = pd.DataFrame({'agent': value_counts, 'Percentage (%)': percentage})

print(result)

       agent  Percentage (%)
agent                       
0      40848       46.838665
2      13027       14.937507
1       8598        9.858961
24      3342        3.832129
22      3294        3.777090
6       2779        3.186561
4       1644        1.885105
26      1493        1.711960
5       1383        1.585827
19      1228        1.408095
14      1116        1.279670
11       986        1.130604
17       844        0.967779
9        722        0.827887
28       614        0.704048
29       523        0.599702
8        477        0.546956
15       402        0.460956
23       394        0.451783
10       363        0.416237
25       344        0.394450
18       312        0.357757
3        302        0.346291
16       281        0.322211
13       277        0.317624
12       272        0.311891
30       257        0.294691
27       223        0.255705
21       220        0.252265
7        216        0.247678
31       215        0.246531
20       214        0.245385


In [383]:
# Outlier remover
def remove_outlier(data, col):
    # Determine IQR for column
    col_values = data[col].values
    q25, q75 = np.percentile(col_values, 25), np.percentile(col_values, 75)
    print('Quartile 25: {} | Quartile 75: {}'.format(q25, q75))
    iqr = q75 - q25
    print('IQR: {}'.format(iqr))
    print()

    # Determine the cut off (IQR * 1.5)
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off
    print('Cut Off: {}'.format(cut_off))
    print('Lower bound: {}'.format(lower))
    print('Upper bound: {}'.format(upper))
    print()

    # Examine instances before removal
    print("Number of instances before removal: {}".format(len(data)))

    # Remove outliers
    data = data.drop(data[(data[col] > upper) | (data[col] < lower)].index)

    # Examine instances after removal
    print("Number of instances after removal: {}".format(len(data)))
    print()

    return data

In [384]:
df.corr()['is_canceled'].sort_values(ascending=False)

is_canceled                       1.000000
reservation_status                0.975277
lead_time                         0.184427
deposit_type                      0.162641
distribution_channel              0.140201
adr                               0.127214
total_orang                       0.098892
arrival_date_year                 0.088031
stays_in_week_nights              0.084089
adults                            0.080172
hotel                             0.072110
children                          0.067151
stays_in_weekend_nights           0.060899
previous_cancellations            0.051494
meal                              0.046072
reserved_room_type                0.039160
market_segment                    0.033986
arrival_date_day_of_month         0.005433
days_in_waiting_list              0.004702
arrival_date_week_number          0.001622
arrival_date_quarter             -0.006784
arrival_date_month               -0.009176
babies                           -0.020639
reservation

In [385]:
df = remove_outlier(df, 'reservation_status')

Quartile 25: 0.0 | Quartile 75: 1.0
IQR: 1.0

Cut Off: 1.5
Lower bound: -1.5
Upper bound: 2.5

Number of instances before removal: 87210
Number of instances after removal: 87210



In [386]:
df = remove_outlier(df, 'lead_time')

Quartile 25: 11.0 | Quartile 75: 125.0
IQR: 114.0

Cut Off: 171.0
Lower bound: -160.0
Upper bound: 296.0

Number of instances before removal: 87210
Number of instances after removal: 84816



In [387]:
df = remove_outlier(df, 'deposit_type')

Quartile 25: 0.0 | Quartile 75: 0.0
IQR: 0.0

Cut Off: 0.0
Lower bound: 0.0
Upper bound: 0.0

Number of instances before removal: 84816
Number of instances after removal: 83956



In [388]:
jumlah_baris = df.shape[0]
print(f"Jumlah baris: {jumlah_baris}")


Jumlah baris: 83956


In [389]:
df = df.drop('reservation_status', axis=1)
df = df.drop('reservation_status_date', axis=1)
df = df.drop('assigned_room_type', axis=1)

# df = df.drop('required_car_parking_spaces', axis=1)
# df = df.drop('agent', axis=1)
# df = df.drop('customer_type', axis=1)

In [390]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Contoh DataFrame
np.random.seed(42)  # Untuk reproducibility

# Pisahkan fitur (X) dan target (y)
X = df.drop(columns=['is_canceled'])  # Semua kolom kecuali 'is_canceled'
y = df['is_canceled']  # Target

# Split data untuk training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Model Random Forest
# model = RandomForestClassifier(random_state=42)
# model.fit(X_train, y_train)

# # Prediksi
# y_pred = model.predict(X_test)

# # Evaluasi
# print("Accuracy:", accuracy_score(y_test, y_pred))
# print("\nClassification Report:\n", classification_report(y_test, y_pred))


In [391]:
# X = df.drop(columns=['is_canceled'])  # Semua kolom kecuali 'is_canceled'
# y = df['is_canceled']  # Target

# # Train model
# model = XGBoost(max_depth=3, learning_rate=0.1, n_estimators=10)
# model.fit(X, y)

## DTL

In [392]:
from DTL import DTL, entropy, gain, max_gain
import math

In [393]:
sample_size = 100000

selected_features = [
    'is_canceled',
    'hotel',
    'lead_time',
    'stays_in_weekend_nights',
    'stays_in_week_nights',
    'adults',
    'market_segment',
    'distribution_channel',
    'customer_type',
    'is_repeated_guest',
    'previous_cancellations',
    'previous_bookings_not_canceled',
    'reserved_room_type',
    'deposit_type',
    'days_in_waiting_list'
]


selected_df = df[selected_features]


In [394]:
selected_df['previous_bookings_not_canceled'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 19, 26, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72], dtype=int64)

In [395]:
selected_df.nunique()

is_canceled                         2
hotel                               2
lead_time                         297
stays_in_weekend_nights            17
stays_in_week_nights               33
adults                              5
market_segment                      8
distribution_channel                5
customer_type                       4
is_repeated_guest                   2
previous_cancellations             11
previous_bookings_not_canceled     73
reserved_room_type                  9
deposit_type                        1
days_in_waiting_list               85
dtype: int64

In [396]:
selected_df.loc[selected_df['lead_time']>30, 'lead_time'] = 'more'

selected_df['lead_time'] = selected_df['lead_time'].apply(lambda x: str(x))

print(selected_df['lead_time'].value_counts())


lead_time
more    49567
0        5889
1        3195
2        1921
3        1695
4        1561
5        1408
6        1295
7        1183
8        1014
12        913
11        894
9         886
10        860
14        817
16        799
17        767
13        757
18        712
15        711
19        678
20        650
28        618
21        615
29        610
22        597
26        589
24        576
23        568
25        546
27        534
30        531
Name: count, dtype: int64


C:\Users\fadhl\AppData\Local\Temp\ipykernel_7248\1946586415.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'more' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  selected_df.loc[selected_df['lead_time']>30, 'lead_time'] = 'more'
C:\Users\fadhl\AppData\Local\Temp\ipykernel_7248\1946586415.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['lead_time'] = selected_df['lead_time'].apply(lambda x: str(x))


In [397]:
selected_df.loc[selected_df['days_in_waiting_list']>3, 'days_in_waiting_list'] = 'more'

selected_df['days_in_waiting_list'] = selected_df['days_in_waiting_list'].apply(lambda x: str(x))

print(selected_df['days_in_waiting_list'].value_counts())

days_in_waiting_list
0       83403
more      536
1          12
2           4
3           1
Name: count, dtype: int64


C:\Users\fadhl\AppData\Local\Temp\ipykernel_7248\1632619817.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'more' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  selected_df.loc[selected_df['days_in_waiting_list']>3, 'days_in_waiting_list'] = 'more'
C:\Users\fadhl\AppData\Local\Temp\ipykernel_7248\1632619817.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['days_in_waiting_list'] = selected_df['days_in_waiting_list'].apply(lambda x: str(x))


In [398]:
selected_df.loc[selected_df['previous_cancelation']>3, 'previous_cancelation'] = 'more'

selected_df['previous_cancelation'] = selected_df['previous_cancelation'].apply(lambda x: str(x))

print(selected_df['previous_cancelation'].value_counts())
selected_df.loc[selected_df['previous_bookings_not_canceled']>3, 'previous_bookings_not_canceled'] = 'more'

selected_df['previous_bookings_not_canceled'] = selected_df['previous_bookings_not_canceled'].apply(lambda x: str(x))

print(selected_df['previous_bookings_not_canceled'].value_counts())

KeyError: 'previous_cancelation'

In [ ]:
selected_df.loc[selected_df['stays_in_week_nights']>7, 'stays_in_week_nights'] = 'more'

selected_df['stays_in_week_nights'] = selected_df['stays_in_week_nights'].apply(lambda x: str(x))

print(selected_df['stays_in_week_nights'].value_counts())

selected_df.loc[selected_df['stays_in_weekend_nights']>7, 'stays_in_weekend_nights'] = 'more'

selected_df['stays_in_weekend_nights'] = selected_df['stays_in_weekend_nights'].apply(lambda x: str(x))

print(selected_df['stays_in_weekend_nights'].value_counts())

C:\Users\fadhl\AppData\Local\Temp\ipykernel_7248\836146052.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['stays_in_week_nights'] = pd.cut(selected_df['stays_in_week_nights'], bins=bins, labels=labels, right=False)
C:\Users\fadhl\AppData\Local\Temp\ipykernel_7248\836146052.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['stays_in_weekend_nights'] = pd.cut(selected_df['stays_in_weekend_nights'], bins=bins, labels=labels, right=False)


In [ ]:
selected_df.loc[selected_df['adults']>3, 'adults'] = 'more'

selected_df['adults'] = selected_df['adults'].apply(lambda x: str(x))

print(selected_df['adults'].value_counts())

C:\Users\fadhl\AppData\Local\Temp\ipykernel_7248\2717885096.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['adults'] = pd.cut(selected_df['adults'], bins=bins, labels=labels, right=False)


In [ ]:
# selected_df.loc[selected_df['market_segment'].isin([1, 4, 5, 6, 7]), 'market_segment'] = 'others'

# selected_df['market_segment'] = selected_df['market_segment'].apply(lambda x: str(x))

# print(selected_df['market_segment'].value_counts())

In [ ]:
# selected_df.loc[selected_df['distribution_channel'].isin([0,1,3,4]), 'distribution_channel'] = 'others'

# selected_df['distribution_channel'] = selected_df['distribution_channel'].apply(lambda x: str(x))

# print(selected_df['distribution_channel'].value_counts())

In [ ]:
# selected_df.loc[selected_df['customer_type'].isin([2,1,3]), 'customer_type'] = 'others'

# selected_df['customer_type'] = selected_df['customer_type'].apply(lambda x: str(x))

# print(selected_df['customer_type'].value_counts())

In [ ]:
# selected_df.loc[selected_df['reserved_room_type'].isin([0,6,7]), 'reserved_room_type'] = 'others'

# selected_df['reserved_room_type'] = selected_df['reserved_room_type'].apply(lambda x: str(x))

# print(selected_df['reserved_room_type'].value_counts())

In [ ]:
for column in selected_df.columns:
    print(f"Value counts for column '{column}':")
    print(selected_df[column].value_counts())
    print()  # Add a newline for better readability

Value counts for column 'is_canceled':
is_canceled
0    61781
1    22175
Name: count, dtype: int64

Value counts for column 'hotel':
hotel
1    51479
0    32477
Name: count, dtype: int64

Value counts for column 'lead_time':
lead_time
more     50098
week     16964
month    16894
Name: count, dtype: int64

Value counts for column 'stays_in_weekend_nights':
stays_in_weekend_nights
very_low    55801
mid         27886
more          269
Name: count, dtype: int64

Value counts for column 'stays_in_week_nights':
stays_in_week_nights
mid         43141
very_low    27597
more        13218
Name: count, dtype: int64

Value counts for column 'adults':
adults
>=3    67884
<=2    16072
Name: count, dtype: int64

Value counts for column 'market_segment':
market_segment
2    50621
3    12937
0    11649
1     4120
5     3712
4      689
7      226
6        2
Name: count, dtype: int64

Value counts for column 'distribution_channel':
distribution_channel
2    66284
0    12641
1     4845
4      181
3       

In [ ]:
# Sample 20,000 rows where is_canceled == 1
sample = selected_df.sample(n=80000, random_state=42)
oversampled_df = sample

In [ ]:
# # # Step 1: Group by 'stays_in_week_nights'
# grouped = sample.groupby('is_canceled')

# # Step 2: Find the maximum group size
# max_count = grouped.size().max()

# # Step 3: Oversample each group to match the maximum group size
# oversampled_df = grouped.apply(lambda x: x.sample(n=max_count, replace=True)).reset_index(drop=True)

In [ ]:
# # Step 1: Group by 'stays_in_week_nights'
# grouped = oversampled_df.groupby('stays_in_weekend_nights')

# # Step 2: Find the maximum group size
# max_count = grouped.size().max()

# # Step 3: Oversample each group to match the maximum group size
# oversampled_df = grouped.apply(lambda x: x.sample(n=max_count, replace=True)).reset_index(drop=True)

In [ ]:
# Shuffle the dataset (optional but recommended)
import random
random.seed(42)  # For reproducibility

sample = pd.DataFrame(oversampled_df)

# Split ratio (e.g., 80% training, 20% testing)
train_ratio = 0.8
train_size = int(len(sample) * train_ratio)

# Split the oversampled_df
train_set = oversampled_df[:train_size]
test_set = oversampled_df[train_size:]

In [ ]:
model = DTL()
x = model.buildTree(train_set, 'is_canceled')


In [ ]:
x_test = test_set
y = model.predict(x_test)

In [ ]:
print(y)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 

In [ ]:
y_pred = x_test['is_canceled']
y_test = y
correct_predictions = sum(y_true == y_pred for y_true, y_pred in zip(y_test, y_pred))
accuracy = correct_predictions / len(y_test)
print(accuracy)

0.7454375


## OK

In [ ]:
class DBScan:

    alpha = 0.4
    beta = 0.6

    def __Init__(self):
        pass

    def numerical_distance(self, X1, X2):
        column_list = X1.columns
        sum = 0
        for col in column_list:
            sum+=((X1[col]-X2[col])**2) 
        return sum**0.5
    
    def categorical_distance(self, X1, X2):
        column_list = X1.columns
        sum = 0
        for col in column_list:
            if (X1[col]!=X2[col]):
                sum+=1
        return sum
    
    def combine_distance(self, alpha,beta, X_num_1, X_num_2, X_cat_1, X_cat_2):
        return alpha*self.numerical_distance(X_num_1,X_num_2)+beta*self.categorical_distance(X_cat_1,X_cat_2)
    
    def split_numerical_categorical(self, X):
        # Split DataFrame into numerical and categorical columns
        numerical_cols = X.select_dtypes(include=['number']).columns
        categorical_cols = X.select_dtypes(include=['object', 'category']).columns

        numerical_data = X[numerical_cols]
        categorical_data = X[categorical_cols]
        return numerical_data, categorical_data
    
    def build_matrix(self, X):
        # Split the data once for efficiency
        X_num, X_cat = self.split_numerical_categorical(X)

        # Precompute the pairwise distance matrix
        n = X.shape[0]
        matrix = np.zeros((n, n))

        for i in range(n):
            for j in range(i, n):  # Only compute upper triangle (matrix[i][j] = matrix[j][i])
                num_distance = np.linalg.norm(X_num.iloc[i] - X_num.iloc[j])
                cat_distance = (X_cat.iloc[i] != X_cat.iloc[j]).sum()
                matrix[i, j] = self.alpha * num_distance + self.beta * cat_distance
                if i != j:
                    matrix[j, i] = matrix[i, j]  # Symmetry

        return matrix
    
    def clustering(self, X, n, d):
        matrix = self.build_matrix(X)
        clusters = {}
        index_visiting = [ i for i in range(len(matrix))]
        temp_visiting = set({0})
        while len(index_visiting)>0:
            if len(temp_visiting)<=0:
                temp_visiting.add(index_visiting[0])
            C = set()
            iterator = iter(temp_visiting)
            while temp_visiting:
                try:
                    i = next(iterator)
                    for j in range(len(matrix[0])):
                        if matrix[i][j]<=d:
                            C.add(j)
                    temp_visiting = temp_visiting.union(C)
                    temp_visiting.remove(i)
                    temp_visiting.intersection_update(index_visiting)
                    index_visiting.remove(i)
                    iterator = iter(temp_visiting)
                except StopIteration:
                    break
            clusters[len(clusters)+1] = C

            for key in list(clusters.keys()):
                if len(clusters[key])<n:
                    clusters.pop(key)

        return clusters

        
            

In [ ]:
selected_features = [
    'hotel',
    'lead_time',
    'stays_in_weekend_nights',
    'stays_in_week_nights',
    'adults',
    'children',
    'babies',
    'market_segment',
    'distribution_channel',
    'customer_type',
    'adr',
    'total_of_special_requests'
]


# Data untuk clustering
data_clustering = df[selected_features]


In [ ]:
x = data_clustering.sample(n=5000, random_state=42)
model = DBScan()
c = model.clustering(x, 30, 2)

KeyboardInterrupt: 

In [ ]:
for i in c.keys():
    print(c[i])


{0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14, 16, 17, 18, 20, 22, 23, 24, 26, 28, 29, 30, 31, 32, 33, 35, 36, 39, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 53, 55, 56, 59, 60, 61, 63, 65, 66, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 84, 87, 88, 92, 93, 94, 95, 96, 97, 99, 100, 101, 102, 105, 107, 108, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 126, 127, 128, 129, 130, 132, 133, 134, 136, 138, 140, 141, 142, 144, 146, 147, 148, 149, 151, 152, 156, 158, 159, 161, 162, 163, 164, 166, 167, 168, 169, 170, 171, 172, 174, 180, 181, 183, 184, 185, 187, 188, 189, 190, 191, 193, 194, 198, 199, 200, 201, 202, 204, 208, 209, 210, 211, 215, 216, 217, 219, 220, 221, 225, 226, 228, 229, 231, 232, 233, 234, 235, 237, 239, 241, 243, 244, 245, 246, 248, 249, 250, 252, 253, 255, 256, 257, 258, 259, 260, 264, 265, 266, 267, 268, 269, 270, 272, 273, 274, 275, 276, 277, 280, 282, 283, 284, 285, 286, 287, 288, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 309, 310